In [1]:
import os
import re
import sys
import colorama
from colorama import Fore, Style
from concurrent.futures import ThreadPoolExecutor, as_completed

print("NOTE - THIS WILL NOT DAMAGE ANY FILES. IT MAY TRIGGER ANTIVIRUS SOFTWARES")
print()

# Initialize Colorama for colored output.
colorama.init(autoreset=True)

# Define programming language signatures using file extensions and content patterns.
LANGUAGE_PATTERNS = {
    'Python': {
        'extensions': ['.py'],
        'patterns': [r'\bdef\b', r'\bimport\b']
    },
    'C++': {
        'extensions': ['.cpp', '.cxx', '.cc', '.hpp', '.h'],
        'patterns': [r'#include', r'\bint\s+main\b']
    },
    'C': {
        'extensions': ['.c'],
        'patterns': [r'#include', r'\bint\s+main\b']
    },
    'Java': {
        'extensions': ['.java'],
        'patterns': [r'public\s+static\s+void\s+main', r'class\s+\w+']
    },
    'JavaScript': {
        'extensions': ['.js'],
        'patterns': [r'\bfunction\b', r'console\.log']
    },
    'Ruby': {
        'extensions': ['.rb'],
        'patterns': [r'\bdef\b', r'\bend\b', r'\bputs\b']
    },
    'PHP': {
        'extensions': ['.php'],
        'patterns': [r'<\?php']
    },
    'HTML': {
        'extensions': ['.html', '.htm'],
        'patterns': [r'<html']
    },
    'CSS': {
        'extensions': ['.css'],
        'patterns': [r'\{', r'\}']
    },
    'Shell': {
        'extensions': ['.sh'],
        'patterns': [r'#!/bin/(bash|sh)']
    },
    'C#': {
        'extensions': ['.cs'],
        'patterns': [r'using\s+System']
    },
    'Swift': {
        'extensions': ['.swift'],
        'patterns': [r'import\s+Foundation']
    },
    'Kotlin': {
        'extensions': ['.kt'],
        'patterns': [r'fun\s+main']
    },
    'Rust': {
        'extensions': ['.rs'],
        'patterns': [r'fn\s+main']
    },
    'Go': {
        'extensions': ['.go'],
        'patterns': [r'package\s+main', r'func\s+main']
    }
}

# Map each language to a specific color.
COLOR_MAP = {
    'Python': Fore.GREEN,
    'C++': Fore.BLUE,
    'C': Fore.CYAN,
    'Java': Fore.MAGENTA,
    'JavaScript': Fore.YELLOW,
    'Ruby': Fore.RED,
    'PHP': Fore.LIGHTMAGENTA_EX,
    'HTML': Fore.LIGHTBLUE_EX,
    'CSS': Fore.LIGHTCYAN_EX,
    'Shell': Fore.LIGHTGREEN_EX,
    'C#': Fore.LIGHTRED_EX,
    'Swift': Fore.LIGHTYELLOW_EX,
    'Kotlin': Fore.LIGHTWHITE_EX,
    'Rust': Fore.WHITE,
    'Go': Fore.LIGHTBLACK_EX
}

def detect_language_for_file(file_path):
    """
    Attempts to detect the programming language of a file using its extension and content.
    Returns the language name if detected; otherwise, returns None.
    """
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()
    possible_languages = []
    
    # Check file extension.
    for language, info in LANGUAGE_PATTERNS.items():
        if ext in info['extensions']:
            possible_languages.append(language)
    
    # Read a snippet of file content.
    try:
        with open(file_path, 'r', encoding='utf-8', errors="ignore") as f:
            content = f.read(1024)
    except Exception:
        return None

    # Check for language-specific patterns in content.
    for language, info in LANGUAGE_PATTERNS.items():
        for pattern in info['patterns']:
            if re.search(pattern, content):
                return language

    # Fallback: if the extension suggests a language, return the first match.
    if possible_languages:
        return possible_languages[0]
    return None

def collect_file_paths(directory, ignore_dirs):
    """
    Recursively collects all file paths from 'directory', ignoring any directory that contains
    any string from 'ignore_dirs'.
    """
    file_paths = []
    for root, dirs, files in os.walk(directory):
        if any(ignored.lower() in root.lower() for ignored in ignore_dirs):
            continue
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

def main():
    # Prompt for directory input and expand tilde if used.
    directory = os.path.expanduser(input("Enter the directory to scan: ").strip())
    if not os.path.isdir(directory):
        print("Invalid directory. Please enter a valid directory path.")
        sys.exit(1)
    
    # Define directories to ignore. Adjust as needed.
    ignore_dirs = []  # e.g., on macOS: ["/System", "/Applications", "/Library"]

    print(f"\nCollecting files from: {directory}")
    file_paths = collect_file_paths(directory, ignore_dirs)
    total_files = len(file_paths)
    print(f"Total files found: {total_files}\n")

    language_counts = {}

    # Process files concurrently.
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(detect_language_for_file, fp): fp for fp in file_paths}
        processed = 0
        for future in as_completed(futures):
            processed += 1
            lang = future.result()
            if lang:
                language_counts[lang] = language_counts.get(lang, 0) + 1
            if processed % 100 == 0 or processed == total_files:
                print(f"Processed {processed}/{total_files} files...", end='\r')

    print(f"\n\nDetected Programming Languages Report:")
    if language_counts:
        for language in sorted(language_counts):
            color = COLOR_MAP.get(language, Fore.WHITE)
            print(f"{color}{language}: {language_counts[language]} files{Style.RESET_ALL}")
    else:
        print("No programming languages detected in the scanned directory.")

# Run the main function.
main()


Enter the directory to scan: /Volumes/T9 1

Total files found: 533710

Processed 533710/533710 files...

Detected Programming Languages Report:
C: 162 files
C#: 42 files
C++: 147963 files
CSS: 124930 files
HTML: 14010 files
Java: 6703 files
JavaScript: 5457 files
Python: 41566 files
Ruby: 1715 files
Shell: 130 files
